In [ ]:



import re
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm

#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load data
import re
import pandas as pd
#data
hadoop = pd.read_csv('/content/drive/MyDrive/hadoop/hadoop.csv')
hadoop['app_id'] = hadoop.Label.str.split('#@#').str[0] 
### remove master node logs
###master_nodes = list(hadoop[hadoop.Content.str.contains('Created MRAppMaster for application')].Label.unique())
###hadoop = hadoop[~hadoop.Label.isin(master_nodes)].reset_index(drop = True) 

# only keep master nodes
master_nodes = list(hadoop[hadoop.Content.str.contains('Created MRAppMaster for application')].Label.unique())
hadoop = hadoop[hadoop.Label.isin(master_nodes)].reset_index(drop = True)

#label
label = []
with open('/content/drive/MyDrive/hadoop/abnormal_label.txt','r') as data:
    for line in data:
        if re.search('application_[a-zA-Z0-9_]+',line):
            label.append(re.search('application_[a-zA-Z0-9_]+',line).group())
        else:
            label.append(line)

label_normal = label[4:7]+label[38:46]
label_normal = [i for i in label_normal if re.search(r'application_[0-9_]*',i)]

label_abnormal = [x for x in label if x not in label_normal]
label_abnormal = [i for i in label_abnormal if re.search(r'application_[0-9_]*',i)]

In [ ]:
# train test split
train_set = label[4:6] + label[42:46]

test_abnormal = label_abnormal

test_normal = [x for x in label_normal if x not in train_set]

train_df = hadoop[hadoop.app_id.isin(train_set)].reset_index(drop = True)
test_abnormal = hadoop[hadoop.app_id.isin(test_abnormal)].reset_index(drop = True)
test_normal = hadoop[hadoop.app_id.isin(test_normal)].reset_index(drop = True)

In [ ]:
#create txt files
def create_file(dataset,save_PATH,f2 = False, Id = 'Id'):
    dataset.sort_values('Time',inplace = True)
    f1 = dataset.groupby(Id)['Template_id'].agg(lambda x: ' '.join([str(i) for i in x])+'\n')
    if f2:
        f2 = dataset.groupby('Id')['diff'].agg(list).apply(lambda x: [i-min(x) for i in x])
        f2 = f2.apply((lambda x: ' '.join([str(i) for i in x])+'\n'))
        dataset = f1+'@@'+f2
    else:
        dataset = f1
    dataset = dataset.to_list()
    with open(save_PATH, 'w') as data:
        for i in dataset:
            data.write(i)

In [ ]:
create_file(train_df,'/content/drive/MyDrive/hadoop/hadoop_train.txt',Id = 'app_id')
create_file(test_abnormal,'/content/drive/MyDrive/hadoop/hadoop_test_abnormal.txt',Id = 'app_id')
create_file(test_normal,'/content/drive/MyDrive/hadoop/hadoop_test_normal.txt',Id = 'app_id')